# Assignment 2

**Credits**: Federico Ruggeri, Eleonora Mancini, Paolo Torroni

**Keywords**: Human Value Detection, Multi-label classification, Transformers, BERT


# Contact

For any doubt, question, issue or help, you can always contact us at the following email addresses:

Teaching Assistants:

* Federico Ruggeri -> federico.ruggeri6@unibo.it
* Eleonora Mancini -> e.mancini@unibo.it

Professor:

* Paolo Torroni -> p.torroni@unibo.it

# Introduction

You are tasked to address the [Human Value Detection challenge](https://aclanthology.org/2022.acl-long.306/).

## Problem definition

Arguments are paired with their conveyed human values.

Arguments are in the form of **premise** $\rightarrow$ **conclusion**.

### Example:

**Premise**: *``fast food should be banned because it is really bad for your health and is costly''*

**Conclusion**: *``We should ban fast food''*

**Stance**: *in favour of*

<center>
    <img src="images/human_values.png" alt="human values" />
</center>

# [Task 1 - 0.5 points] Corpus

Check the official page of the challenge [here](https://touche.webis.de/semeval23/touche23-web/).

The challenge offers several corpora for evaluation and testing.

You are going to work with the standard training, validation, and test splits.

#### Arguments
* arguments-training.tsv
* arguments-validation.tsv
* arguments-test.tsv

#### Human values
* labels-training.tsv
* labels-validation.tsv
* labels-test.tsv

### Example

#### arguments-*.tsv
```

Argument ID    A01005

Conclusion     We should ban fast food

Stance         in favor of

Premise        fast food should be banned because it is really bad for your health and is costly.
```

#### labels-*.tsv

```
Argument ID                A01005

Self-direction: thought    0
Self-direction: action     0
...
Universalism: objectivity: 0
```

### Splits

The standard splits contain

   * **Train**: 5393 arguments
   * **Validation**: 1896 arguments
   * **Test**: 1576 arguments

### Annotations

In this assignment, you are tasked to address a multi-label classification problem.

You are going to consider **level 3** categories:

* Openness to change
* Self-enhancement
* Conversation
* Self-transcendence

**How to do that?**

You have to merge (**logical OR**) annotations of level 2 categories belonging to the same level 3 category.

**Pay attention to shared level 2 categories** (e.g., Hedonism). $\rightarrow$ [see Table 1 in the original paper.](https://aclanthology.org/2022.acl-long.306/)

#### Example

```
Self-direction: thought:    0
Self-direction: action:     1
Stimulation:                0
Hedonism:                   1

Openess to change           1
```

### Instructions

* **Download** the specificed training, validation, and test files.
* **Encode** split files into a pandas.DataFrame object.
* For each split, **merge** the arguments and labels dataframes into a single dataframe.
* **Merge** level 2 annotations to level 3 categories.

In [254]:
# file management
import urllib
from pathlib import Path

# dataframe management
import pandas as pd

# data manipulation
import numpy as np

# for readability
from typing import Iterable
from tqdm import tqdm

from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report

In [255]:
# ! pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from keras import layers

#### Download

In [256]:
class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_url(download_path: Path, url: str):
    with DownloadProgressBar(unit='B', unit_scale=True,
                             miniters=1, desc=url.split('/')[-1]) as t:
        urllib.request.urlretrieve(url, filename=download_path, reporthook=t.update_to)


def download_dataset(download_path: Path, url: str):
    print("Downloading dataset...")
    download_url(url=url, download_path=download_path)
    print("Download complete!")

def clean_download(url, name, folder):
    print(f"Current work directory: {Path.cwd()}")
    dataset_folder = Path.cwd().joinpath("Datasets").joinpath(folder)

    if not dataset_folder.exists():
        dataset_folder.mkdir(parents=True)

    dataset_path = dataset_folder.joinpath(name)

    if not dataset_path.exists():
        download_dataset(dataset_path, url)

In [257]:
arguments_split = ["arguments-training.tsv", "arguments-validation.tsv", "arguments-test.tsv"]
labels_split = ["labels-training.tsv", "labels-validation.tsv", "labels-test.tsv"]

for argument in arguments_split:
    url = "https://zenodo.org/records/8248658/files/" + argument + "?download=1"
    clean_download(url, argument, "Arguments")

for label in labels_split:
    url = "https://zenodo.org/records/8248658/files/" + label + "?download=1"
    clean_download(url, label, "Labels")


Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers
Current work directory: /Users/giacomopiergentili/Developer/NLP/Assignment2/NLP_multi-label-text-classification-with-transformers


#### Encode

In [258]:
# Read the arguments split files into DataFrames
arguments_train_df = pd.read_table('Datasets/Arguments/arguments-training.tsv', sep='\t')
arguments_val_df = pd.read_table('Datasets/Arguments/arguments-validation.tsv', sep='\t')
arguments_test_df = pd.read_table('Datasets/Arguments/arguments-test.tsv', sep='\t')

# Read the labels split files into DataFrames
labels_train_df = pd.read_table('Datasets/Labels/labels-training.tsv', sep='\t')
labels_val_df = pd.read_table('Datasets/Labels/labels-validation.tsv', sep='\t')
labels_test_df = pd.read_table('Datasets/Labels/labels-test.tsv', sep='\t')


#### Merge arguments and labels

In [259]:
train_merge_df = pd.merge(arguments_train_df, labels_train_df, on='Argument ID')
val_merge_df = pd.merge(arguments_val_df, labels_val_df, on='Argument ID')
test_merge_df = pd.merge(arguments_test_df, labels_test_df, on='Argument ID')


#### Merge level 2 annotations to level 3 categories

* Openness to change
* Self-enhancement
* Conversation
* Self-transcendence

In [260]:
level_2_categories = train_merge_df.columns[4:]

In [261]:
level_2_to_Openness_to_change = level_2_categories[:4]
level_2_to_Self_enhancement = level_2_categories[3:8]
level_2_to_Conservation = level_2_categories[7:14]
level_2_to_Self_transcendence = level_2_categories[13:]

In [262]:
def merge_categories(df):
    df['Openness to change'] = [int(any(df[level_2_to_Openness_to_change].loc[i])) for i in range(len(df))]
    df['Self-enhancement'] = [int(any(df[level_2_to_Self_enhancement].loc[i])) for i in range(len(df))]
    df['Conservation'] = [int(any(df[level_2_to_Conservation].loc[i])) for i in range(len(df))]
    df['Self-transcendence'] = [int(any(df[level_2_to_Self_transcendence].loc[i])) for i in range(len(df))]
    return df.drop(level_2_categories, axis=1)

In [263]:
final_train_df = merge_categories(train_merge_df)
final_val_df = merge_categories(val_merge_df)
final_test_df = merge_categories(test_merge_df)

In [264]:
final_train_df.head()

,Argument ID,Conclusion,Stance,Premise,Openness to change,Self-enhancement,Conservation,Self-transcendence
0,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...,0,0,1,0
1,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...,0,0,1,0
2,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...,0,1,1,0
3,A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...,0,0,1,1
4,A01008,We should ban factory farming,against,factory farming allows for the production of c...,0,0,1,1


# [Task 2 - 2.0 points] Model definition

You are tasked to define several neural models for multi-label classification.

<center>
    <img src="images/model_schema.png" alt="model_schema" />
</center>

### Instructions

* **Baseline**: implement a random uniform classifier (an individual classifier per category).
* **Baseline**: implement a majority classifier (an individual classifier per category).

<br/>

* **BERT w/ C**: define a BERT-based classifier that receives an argument **conclusion** as input.
* **BERT w/ CP**: add argument **premise** as an additional input.
* **BERT w/ CPS**: add argument premise-to-conclusion **stance** as an additional input.

### Notes

**Do not mix models**. Each model has its own instructions.

You are **free** to select the BERT-based model card from huggingface.

#### Examples

```
bert-base-uncased
prajjwal1/bert-tiny
distilbert-base-uncased
roberta-base
```

### BERT w/ C

<center>
    <img src="images/bert_c.png" alt="BERT w/ C" />
</center>

### BERT w/ CP

<center>
    <img src="images/bert_cp.png" alt="BERT w/ CP" />
</center>

### BERT w/ CPS

<center>
    <img src="images/bert_cps.png" alt="BERT w/ CPS" />
</center>

### Input concatenation

<center>
    <img src="images/input_merging.png" alt="Input merging" />
</center>

### Notes

The **stance** input has to be encoded into a numerical format.

You **should** use the same model instance to encode **premise** and **conclusion** inputs.

### Instructions

* **Baseline**: implement a random uniform classifier (an individual classifier per category).
* **Baseline**: implement a majority classifier (an individual classifier per category).

<br/>

* **BERT w/ C**: define a BERT-based classifier that receives an argument **conclusion** as input.
* **BERT w/ CP**: add argument **premise** as an additional input.
* **BERT w/ CPS**: add argument premise-to-conclusion **stance** as an additional input.

#### Text encoding

First, we are going to use ``datasets`` library to encode our dataset into a handy wrapper for computational speedup.

In [265]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

def encode_stance(df):
    return le.fit_transform(df['Stance'])

final_train_df['Stance'] = encode_stance(final_train_df)
final_val_df['Stance'] = encode_stance(final_val_df)
final_test_df['Stance'] = encode_stance(final_test_df)

/opt/homebrew/Caskroom/miniconda/base/envs/nlp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/Caskroom/miniconda/base/envs/nlp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/homebrew/Caskroom/miniconda/base/envs/nlp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/Caskroom/miniconda/base/envs/nlp-env/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is depreca

In [266]:
final_train_df.head()

,Argument ID,Conclusion,Stance,Premise,Openness to change,Self-enhancement,Conservation,Self-transcendence
0,A01002,We should ban human cloning,1,we should ban human cloning as it will only ca...,0,0,1,0
1,A01005,We should ban fast food,1,fast food should be banned because it is reall...,0,0,1,0
2,A01006,We should end the use of economic sanctions,0,sometimes economic sanctions are the only thin...,0,1,1,0
3,A01007,We should abolish capital punishment,0,capital punishment is sometimes the only optio...,0,0,1,1
4,A01008,We should ban factory farming,0,factory farming allows for the production of c...,0,0,1,1


In [267]:
from datasets import Dataset

train_data = Dataset.from_pandas(final_train_df)
validation_data = Dataset.from_pandas(final_val_df)
test_data = Dataset.from_pandas(final_test_df)

In [268]:
print(train_data)
print(validation_data)
print(test_data)

Dataset({
    features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence'],
    num_rows: 5393
})
Dataset({
    features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence'],
    num_rows: 1896
})
Dataset({
    features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence'],
    num_rows: 1576
})


Transformers typically use [SentencePiece tokenizer](https://github.com/google/sentencepiece) to perform sub-word level tokenization.

In particular, the `transformers` library offers the `AutoTokenizer` class to quickly retrieve our chosen transformer's ad-hoc tokenizer.

I chose bert-tiny as first cause it's lighter and was suggested by the teacher, but we can change it with the others if needed

In [269]:
from transformers import AutoTokenizer

model_card = 'prajjwal1/bert-tiny'

tokenizer = AutoTokenizer.from_pretrained(model_card)

The `model_card` variable defines the *path* where to look for our pre-trained model.

You can check [huggingface's hub](https://huggingface.co/models) model hub to pick the model card according to your preference.

In [270]:
# pad = max(max([len(phrase) for phrase in train_data['Premise']]), max([len(phrase['Premise']) + len(phrase['Conclusion']) for phrase in validation_data]), max([len(phrase['Premise']) + len(phrase['Conclusion']) for phrase in test_data]))
pad = max([len(phrase) for phrase in train_data['Premise']])+max([len(phrase) for phrase in train_data['Conclusion']])
print(pad)

982


In [271]:
def preprocess_premise_conclusion(texts):
    return tokenizer(texts['Premise'], texts['Conclusion'], padding=True, max_length=pad ,  truncation=True) 

train_data = train_data.map(preprocess_premise_conclusion, batched=True)
validation_data = validation_data.map(preprocess_premise_conclusion, batched=True)
test_data = test_data.map(preprocess_premise_conclusion, batched=True)

In [272]:
print(train_data)
print(validation_data)
print(test_data)

Dataset({
    features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5393
})
Dataset({
    features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1896
})
Dataset({
    features: ['Argument ID', 'Conclusion', 'Stance', 'Premise', 'Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1576
})


In [273]:
print(train_data['input_ids'][50])

[101, 4352, 2336, 2000, 11839, 1998, 2202, 2070, 4251, 2051, 1999, 2082, 2064, 2022, 15189, 2005, 2037, 5177, 2740, 1012, 102, 2057, 2323, 23469, 2082, 7083, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [274]:
print(train_data['attention_mask'][50])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [275]:
original_text = train_data['Premise'][50] + ' ' + train_data['Conclusion'][50]
decoded_text = tokenizer.decode(train_data['input_ids'][50])

print(original_text + "\n\n" + decoded_text)

allowing children to pray and take some quiet time in school can be beneficial for their mental health. We should prohibit school prayer

[CLS] allowing children to pray and take some quiet time in school can be beneficial for their mental health. [SEP] we should prohibit school prayer [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [276]:
labels = ['Openness to change', 'Self-enhancement', 'Conservation', 'Self-transcendence']
num_labels = len(labels)

In [277]:
id2label = {str(i):label for i, label in enumerate(labels)}
label2id = {label:str(i) for i, label in enumerate(labels)}

In [278]:
print(id2label)
print(label2id)

{'0': 'Openness to change', '1': 'Self-enhancement', '2': 'Conservation', '3': 'Self-transcendence'}
{'Openness to change': '0', 'Self-enhancement': '1', 'Conservation': '2', 'Self-transcendence': '3'}


#### Implementing a random uniform classifier for each category

In [279]:
otc_uniform_classifier = DummyClassifier(strategy="uniform")
se_uniform_classifier = DummyClassifier(strategy="uniform")
cons_uniform_classifier = DummyClassifier(strategy="uniform")
st_uniform_classifier = DummyClassifier(strategy="uniform")

#### Implementing a majority classifier for each category

In [280]:
otc_majority_classifier = DummyClassifier(strategy="prior")
se_majority_classifier = DummyClassifier(strategy="prior")
cons_majority_classifier = DummyClassifier(strategy="prior")
st_majority_classifier = DummyClassifier(strategy="prior")

#### Defining a BERT-based classifier that receives an argument **conclusion** as input.

We first need to format input data to be fed as mini-batches in a training/evaluation procedure.<br>
https://discuss.huggingface.co/t/whats-the-input-of-bert/14932

In [281]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [282]:
from transformers import AutoModelForSequenceClassification

c_model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                             num_labels=num_labels,
                                                             id2label=id2label,
                                                             label2id=label2id)

cp_model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                             num_labels=num_labels,
                                                             id2label=id2label,
                                                             label2id=label2id)

cps_model = AutoModelForSequenceClassification.from_pretrained(model_card,
                                                             num_labels=num_labels,
                                                             id2label=id2label,
                                                             label2id=label2id)

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initia

In [283]:
print(c_model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

In [293]:
import torch

#fake_input = torch.tensor([0 for i in range(pad)])
# fake_input[0].size

#c_model(fake_input)

c_model(input_ids= train_data['input_ids'][1], attention_mask = train_data['attention_mask'][1])


AttributeError: 'list' object has no attribute 'size'

In [ ]:
# output = c_model(train_data["input_ids"][1])
# cmq per ora si farei padding su tutto almeno da vedere se va. perchè in generale credo che il problema sia nel cambio di dim dellinput
print(train_data["input_ids"][1])
print(tokenizer.decode(train_data["input_ids"][1]))

[101, 3435, 2833, 2323, 2022, 7917, 2138, 2009, 2003, 2428, 2919, 2005, 2115, 2740, 1998, 2003, 17047, 1012, 102, 2057, 2323, 7221, 3435, 2833, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[CLS] fast food should be banned because it is really bad for your health and is costly. [SEP] we should ban fast food [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


# [Task 3 - 0.5 points] Metrics

Before training the models, you are tasked to define the evaluation metrics for comparison.

### Instructions

* Evaluate your models using per-category binary F1-score.
* Compute the average binary F1-score over all categories (macro F1-score).

### Example

You start with individual predictions ($\rightarrow$ samples).

```
Openess to change:    0 0 1 0 1 1 0 ...
Self-enhancement:     1 0 0 0 1 0 1 ...
Conversation:         0 0 0 1 1 0 1 ...
Self-transcendence:   1 1 0 1 0 1 0 ...
```

You compute per-category binary F1-score.

```
Openess to change F1:    0.35
Self-enhancement F1:     0.55
Conversation F1:         0.80
Self-transcendence F1:   0.21
```

You then average per-category scores.
```
Average F1: ~0.48
```

# [Task 4 - 1.0 points] Training and Evaluation

You are now tasked to train and evaluate **all** defined models.

### Instructions

* Train **all** models on the train set.
* Evaluate **all** models on the validation set.
* Pick **at least** three seeds for robust estimation.
* Compute metrics on the validation set.
* Report **per-category** and **macro** F1-score for comparison.

# [Task 5 - 1.0 points] Error Analysis

You are tasked to discuss your results.

### Instructions

* **Compare** classification performance of BERT-based models with respect to baselines.
* Discuss **difference in prediction** between the best performing BERT-based model and its variants.

### Notes

You can check the [original paper](https://aclanthology.org/2022.acl-long.306/) for suggestions on how to perform comparisons (e.g., plots, tables, etc...).

# [Task 6 - 1.0 points] Report

Wrap up your experiment in a short report (up to 2 pages).

### Instructions

* Use the NLP course report template.
* Summarize each task in the report following the provided template.

### Recommendations

The report is not a copy-paste of graphs, tables, and command outputs.

* Summarize classification performance in Table format.
* **Do not** report command outputs or screenshots.
* Report learning curves in Figure format.
* The error analysis section should summarize your findings.

# Submission

* **Submit** your report in PDF format.
* **Submit** your python notebook.
* Make sure your notebook is **well organized**, with no temporary code, commented sections, tests, etc...
* You can upload **model weights** in a cloud repository and report the link in the report.

# FAQ

Please check this frequently asked questions before contacting us

### Model card

You are **free** to choose the BERT-base model card you like from huggingface.

### Model architecture

You **should not** change the architecture of a model (i.e., its layers).

However, you are **free** to play with their hyper-parameters.

### Model Training

You are **free** to choose training hyper-parameters for BERT-based models (e.g., number of epochs, etc...).

### Neural Libraries

You are **free** to use any library of your choice to address the assignment (e.g., Keras, Tensorflow, PyTorch, JAX, etc...)

### Error Analysis

Some topics for discussion include:
   * Model performance on most/less frequent classes.
   * Precision/Recall curves.
   * Confusion matrices.
   * Specific misclassified samples.

# The End